* 진형님 LDA 참고하여 진행
* 동현님 gensim 파일 참고하여 진행

# 1. 데이터 로드

저장된 데이터 가져오기

In [1]:
import pandas as pd
import re
import os

In [2]:
import matplotlib.pyplot as plt

# 한글 깨짐 방지
plt.rcParams['font.family'] = "Malgun Gothic"

# 마이너스 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

In [3]:
# 파일 불러오기 
current_dir = os.path.dirname(os.path.abspath('df_labor_real_cleaned3.csv'))
data_dir = os.path.join(current_dir, '..', 'data')
file_path = os.path.join(data_dir, 'df_labor_real_cleaned3.csv')  # 실제 파일명으로 변경

# CSV 파일 읽기
data = pd.read_csv(file_path)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5016 entries, 0 to 5015
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   판례정보일련번호      5016 non-null   int64 
 1   사건번호          5016 non-null   object
 2   선고            5016 non-null   object
 3   판결유형          5016 non-null   object
 4   판시사항          3725 non-null   object
 5   판결요지          3054 non-null   object
 6   참조조문          3724 non-null   object
 7   참조판례          2450 non-null   object
 8   판례내용          5016 non-null   object
 9   판례내용_상단       5016 non-null   object
 10  판례내용_이유       5012 non-null   object
 11  판례내용_이유(전처리)  5016 non-null   object
 12  판례내용_제거(불용어)  5016 non-null   object
 13  판례내용_이유(불용어)  5012 non-null   object
dtypes: int64(1), object(13)
memory usage: 548.8+ KB


In [9]:
df_labor = data.copy()
df_labor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5016 entries, 0 to 5015
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   판례정보일련번호      5016 non-null   int64 
 1   사건번호          5016 non-null   object
 2   선고            5016 non-null   object
 3   판결유형          5016 non-null   object
 4   판시사항          3725 non-null   object
 5   판결요지          3054 non-null   object
 6   참조조문          3724 non-null   object
 7   참조판례          2450 non-null   object
 8   판례내용          5016 non-null   object
 9   판례내용_상단       5016 non-null   object
 10  판례내용_이유       5012 non-null   object
 11  판례내용_이유(전처리)  5016 non-null   object
 12  판례내용_제거(불용어)  5016 non-null   object
 13  판례내용_이유(불용어)  5012 non-null   object
dtypes: int64(1), object(13)
memory usage: 548.8+ KB


# 2. 데이터 전처리

## 2-1 토큰화

* 형태소 분석 및 전처리: Okt를 사용하여 한국어 문서에서 명사만을 추출하고, 불용어를 제거합니다. 또한, 한 글자 단어들은 제거하여 의미가 있는 단어들만 남깁니다.
* 단어 사전 및 코퍼스 생성: Dictionary 객체를 사용해 단어 사전을 만들고, 이를 통해 문서 코퍼스를 bag-of-words 형식으로 변환합니다.
* LDA 모델 학습: 주제 수(num_topics)를 설정한 후 LDA 모델을 학습합니다.
주제별 핵심 단어 출력: print_topics 메서드를 사용하여 각 주제의 상위 단어를 출력합니다.

In [13]:
# 토픽수 5개
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

# 전처리 수행
df_labor.fillna('',inplace =True)  # NaN 값 제거

# CountVectorizer로 단어 빈도 행렬 생성
vectorizer = CountVectorizer(max_df=0.95, min_df=2)  # 불용어 제거 설정
X = vectorizer.fit_transform(df_labor)

# LDA 모델 생성 및 학습
n_topics = 6  # 원하는 토픽 개수
lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
lda.fit(X)

# 주제별 상위 단어 출력
terms = vectorizer.get_feature_names_out()
for idx, topic in enumerate(lda.components_):
    print(f"\n주제 {idx + 1}:")
    print([terms[i] for i in topic.argsort()[:-10 - 1:-1]])  # 상위 10개 단어 출력


주제 1:
['지급', '임금', '근로', '수당', '퇴직금', '소득', '금액', '회사', '퇴직', '근로자']

주제 2:
['징계', '회사', '노동조합', '해고', '노조', '근로자', '단체', '노동', '쟁의', '조합원']

주제 3:
['공무원', '임용', '교원', '국가', '학교', '활동', '교육', '사회', '정보', '거부']

주제 4:
['근로자', '사업', '근로', '계약', '고용', '노동조합', '임금', '근무', '업체', '파견']

주제 5:
['회사', '주식회사', '사업', '주식', '거래', '순번', '명의', '법인', '계좌', '자금']

주제 6:
['보험', '재해', '급여', '망인', '사고', '지급', '장해', '손해', '산업', '병원']


# 3. 계층적 군집분석

1. 데이터 준비
계층적 군집화를 수행할 텍스트 데이터를 준비합니다. 이 단계에서 데이터 정제, 토큰화, 불용어 제거 등의 전처리 과정을 수행합니다.

2. 텍스트 벡터화
문서나 텍스트 데이터를 수치적으로 표현하기 위해 벡터화합니다. 주로 사용되는 방법은 TF-IDF(Term Frequency-Inverse Document Frequency) 또는 Word Embedding 기법(예: Word2Vec, FastText)입니다.

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 예시 문서 리스트
documents = df_labor['판례내용_이유(불용어)']

# TF-IDF 벡터화
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(documents)


3. 거리 또는 유사성 측정
각 문서 간의 거리 또는 유사성을 측정합니다. 일반적으로 코사인 유사도, 유클리드 거리 등을 사용합니다.

In [23]:
from sklearn.metrics.pairwise import cosine_similarity

# 코사인 유사도 계산
similarity_matrix = cosine_similarity(X)

4. 계층적 군집화 수행
SciPy 라이브러리의 linkage 및 dendrogram 함수를 사용하여 계층적 군집화를 수행하고 덴드로그램을 시각화합니다.

In [ ]:
import numpy as np
import scipy.cluster.hierarchy as sch
import matplotlib.pyplot as plt

# 거리 행렬 계산
from scipy.spatial.distance import pdist

value = df.get('column_name', default_value)  # column_name이 없으면 default_value 반환

# 거리 행렬
distance_matrix = pdist(X.toarray(), metric='euclidean')

# 계층적 군집화
Z = sch.linkage(distance_matrix, method='ward')

# 덴드로그램 시각화
plt.figure(figsize=(10, 5))
dendrogram = sch.dendrogram(Z, labels=documents)
plt.title('Dendrogram')
plt.xlabel('Documents')
plt.ylabel('Distance')
plt.show()

KeyError: -4297

<Figure size 1000x500 with 0 Axes>

5. 군집 결과 해석
덴드로그램을 해석하여 최적의 클러스터 개수를 선택하고, 이를 기반으로 데이터를 그룹화합니다. 각 클러스터의 문서들을 확인하여 그룹의 특성을 분석할 수 있습니다.

6. 클러스터링 결과 평가
클러스터링의 품질을 평가하기 위해 실루엣 점수(silhouette score)나 Davies–Bouldin 지수를 사용할 수 있습니다.

In [20]:
from sklearn.metrics import silhouette_score

# 군집화 결과에 대한 실루엣 점수
labels = sch.fcluster(Z, t=2, criterion='maxclust')  # 2개의 클러스터로 나누기
score = silhouette_score(X, labels)
print(f'Silhouette Score: {score}')

ValueError: Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)

In [19]:
labels = sch.fcluster(Z, t=3, criterion='maxclust')  # 클러스터 수를 3으로 변경

In [25]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.cluster.hierarchy import dendrogram, linkage
import matplotlib.pyplot as plt


# NaN 값 처리 (빈 문자열로 대체 또는 제거)
df_labor['판례내용_이유(불용어)'].fillna('', inplace=True)  # NaN을 빈 문자열로 대체
# df.dropna(subset=['case_text'], inplace=True)  # NaN을 포함한 행 제거


# TF-IDF 벡터화
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_labor['판례내용_이유(불용어)'].iloc[:201])

# 계층적 군집 분석
linkage_matrix = linkage(X.toarray(), method='ward')

# 덴드로그램 시각화
plt.figure(figsize=(10, 7))
dendrogram(linkage_matrix, labels=data['판례내용_이유(불용어)'].astype(str))
plt.title('Dendrogram for Case Law Clustering')
plt.xlabel('Case ID')
plt.ylabel('Distance')
plt.show()

ValueError: Dimensions of Z and labels must be consistent.

<Figure size 1000x700 with 0 Axes>

In [12]:
print(df.head())  # 데이터프레임의 처음 5행 출력
print(df.columns)  # 데이터프레임의 열 이름 출력
print(df.index)    # 데이터프레임의 인덱스 출력

   판례정보일련번호                         사건번호  선고 판결유형  \
0    241221                   2024두32973  선고   판결   
1    241233  2024다211908, 211915, 211922  선고   판결   
2    241219                  2020다287921  선고   판결   
3    241051                  2021다274069  선고   판결   
4    241027                  2023다217312  선고   판결   

                                                판시사항  \
0   [1] 부당해고 등 구제절차에서 피신청인의 추가·변경이 허용되는지 여부(한정 적극...   
1   [1] 파견근로자 보호 등에 관한 법률에 따라 직접고용의무가 발생하였으나 사용사업...   
2   구 파견근로자 보호 등에 관한 법률에 따라 직접고용간주의 효과가 발생하였으나 사용...   
3   [1] 원고용주가 근로자로 하여금 제3자를 위한 업무를 수행하도록 하는 경우, 파...   
4   [1] 대학의 시간강사가 근로기준법 제18조 제3항에서 정한 ‘초단시간근로자’에 ...   

                                                판결요지  \
0   [1] 부당해고 등 구제절차에서 최초 구제신청의 대상이 된 불이익처분을 다투는 범...   
1   [1] 파견근로자 보호 등에 관한 법률에 따라 직접고용의무가 발생하였으나 사용사업...   
2   구 파견근로자 보호 등에 관한 법률(2006. 12. 21. 법률 제8076호로 ...   
3   [1] 원고용주가 어느 근로자로 하여금 제3자를 위한 업무를 수행하도록 하는 경우...   
4   [1] 근로기준법 제18조는 1주 동안의 소정근로시간이 그 사업장에서 같은 종류의...   

       

In [13]:
key_to_check = -4297
if key_to_check in df.index:
    # 키가 존재할 경우 실행할 코드
    print(df.loc[key_to_check])
else:
    print(f"Key {key_to_check} does not exist in the DataFrame.")



Key -4297 does not exist in the DataFrame.
